In [96]:
import pandas as pd
import numpy as np
import networkx as nx

import os
import rich

# path to current directory
path = os.getcwd()

## Import data

Data import from earlier prepared dataframe containing usernames and movie titles.

In [78]:
net_data = pd.read_csv(os.path.join(path, "net_data.csv"))

# removing duplicates if any
net_data.drop_duplicates(inplace = True)
net_data.reset_index(drop = True, inplace = True)

net_data.sample(5)

,username,title
94904,charlie-thulin,Casino Royale
479006,StarWarsMoviesSuck,The Expendables 2
224981,Marc_parry,Interstellar
54743,gerddeblieck,Barry Lyndon
624422,evanhansrote,United 93


## Network preparation

### Username

Prepare username nodes and movie nodes for network. The nodes for this group will contain:

    - user name ["username"]
    - number of movies this user has reviewed ["revs_num"]

In [94]:
# count values of each user's reviews and save outcome to dataframe
user_nodes = net_data.username.value_counts().to_frame(name = "revs_num")
user_nodes.reset_index(inplace = True)

# change column name that was affected by index reset
user_nodes = user_nodes.rename(columns = {"index" : "username"})

Relation containing nodes and their attributes, alongside some basic measures is shown below.

In [118]:
rich.print(f"\n\n[bold]Total number of username nodes: {len(user_nodes.index)}")
user_nodes.head(5)

Total number of username nodes: 309497

,username,revs_num
0,SnoopyStyle,815
1,jboothmillard,717
2,TxMike,642
3,anaconda-40658,589
4,bob the moo,578


## Create network graph with use of NetworkX

Graph structure was chosen to be directed, since reviews are going from user to specific movie.

In [14]:
# initiate empty graph structure
G = nx.DiGraph()